In [1]:
import js
from io import StringIO
import pandas as pd
import datetime
import numpy as np

In [2]:
op = js.self.indexedDB.open("JupyterLite Storage")

In [3]:
idbdb = op.result

In [4]:
tr  = idbdb.transaction("files")
obs = tr.objectStore("files")
req = obs.get("energy_data.csv", "key")

In [15]:
#zeros are transform into nulls, column reordered and renamed, rolling mean to 0 values in energy consumption columns
csv = req.result.content
csv_io = StringIO(csv)
df = pd.read_csv(csv_io)
n = 3
Consumption_2001_mean = (df['energy_consumpt_2001'].replace(0, np.nan)
                         .rolling(n * 2 + 1, min_periods=1, center=True)
                         .mean())
df.loc[df['energy_consumpt_2001'] == 0, 'energy_consumpt_2001'] = Consumption_2001_mean
Consumption_2002_mean = (df['energy_consumpt_2002'].replace(0, np.nan)
                         .rolling(n * 2 + 1, min_periods=1, center=True)
                         .mean())
df.loc[df['energy_consumpt_2002'] == 0, 'energy_consumpt_2002'] = Consumption_2002_mean
df['energy_consumpt_2002'] = np.where(df['Hour'] > 8760, np.nan, df['energy_consumpt_2002'])
cols = ['date', 'energy_consumption_2000', 'energy_consumption_2001'
        ,'temperature_2000', 'humidity_2000',  'temperature_2001','humidity_2001']
df.columns = cols
df = df[['date', 'energy_consumption_2000', 'energy_consumption_2001',
         'temperature_2000', 'temperature_2001', 'humidity_2000', 'humidity_2001']]

In [16]:
#hour column value is changed into date m-d H:M
date_col = pd.date_range("2000-01-01", periods=8784, freq="H")
df['date'] = date_col.values

In [17]:
#extract from date hour, day, week, groups data
newdf = df
#newdf['week'] = newdf.date.dt.isocalendar().week
newdf['hour']  = newdf.date.dt.hour
newdf['day']   = newdf.date.dt.day
newdf['week']  = newdf.date.dt.week
newdf["hour"]  = newdf["date"].apply(lambda x: x.hour)
newdf["day"]   = newdf["date"].apply(lambda x: x.day)
newdf["week"]  = newdf["date"].apply(lambda x: x.week)

<ipython-input-17-8c1575c37154>:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  newdf['week']  = newdf.date.dt.week


In [18]:
#calculates all metrics on a hour, day, week level
energy_hourly = newdf.groupby(['hour']).mean().drop(columns=['day' ,'week'])
energy_daily  = newdf.groupby(['day' ]).mean().drop(columns=['hour','week'])
energy_weekly = newdf.groupby(['week']).mean().drop(columns=['hour','day'])

In [19]:
#energy_hourly.describe()
energy_daily.describe()
#energy_weekly.describe()

,energy_consumption_2000,energy_consumption_2001,temperature_2000,temperature_2001,humidity_2000,humidity_2001
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,489.191502,769.740850,17.608299,18.239584,62.110989,61.510264
std,9.272465,11.820801,0.527684,0.903852,3.967177,2.201699
min,472.394424,747.925187,16.280903,16.332292,54.684028,53.333333
25%,483.618759,761.268918,17.287326,17.685590,59.157986,60.442708
50%,486.909792,768.645313,17.702431,18.302778,61.517361,61.531250
75%,492.281968,780.023350,17.948785,18.738889,64.328125,62.800347
max,515.926564,800.041603,18.773611,20.634722,70.118056,64.809028
